## INFO 371 Problem Set 6

*Name*: Xiaobing Xu
    
*Section*: INFO 371

*Date*: May 22nd, 2022

In [1]:
import pandas as pd
import numpy as np

### 1.1

In [2]:
emails = pd.read_csv("../Data/all-data/lingspam-emails.csv.bz2",sep='\t')
emails.head()

,spam,files,message
0,False,3-1msg1.txt,Subject: re : 2 . 882 s - > np np > date : su...
1,False,3-1msg2.txt,Subject: s - > np + np the discussion of s - ...
2,False,3-1msg3.txt,Subject: 2 . 882 s - > np np . . . for me it ...
3,False,3-375msg1.txt,"Subject: gent conference "" for the listserv ""..."
4,False,3-378msg1.txt,Subject: query : causatives in korean could a...


### 1.2

In [3]:
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(binary=True)
# define vectorizer
X = vectorizer.fit_transform(emails.message)
Y = emails.spam.values
# vectorize your data. Note: this creates a sparse matrix, # use .toarray() if you run into trouble
vocabulary = vectorizer.get_feature_names()
# in case you want to see what are the actual words

In [4]:
DTM = pd.DataFrame(X.toarray(), columns=vocabulary)

### 1.3

In [5]:
from sklearn.model_selection import train_test_split

Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size = 0.2)

### 1.4

- `P_S1`: Pr(category = S)
- `P_S0`: Pr(category = NS)
- `P_W1`: Pr(word = 1)
- `P_W0`: Pr(word = 0)
- `P_W1_S1`: Pr(word = 1 | category = S)
- `P_W0_S1`: Pr(word = 0 | category = S)
- `P_W1_S0`: Pr(word = 1 | category = NS)
- `P_W0_S0`: Pr(word = 0 | category = NS)

### 2.1

In [7]:
##### a. emails correstponding to
emails.message.values[982:985]

array(['Subject: summary : parsing of ambiguous sequences in v2 languages  below is a list of references that i received in response to my query about the parsing of ambiguous svo / ovs sequences in v2 languages . many thanks to the following linguists for their quick and informative replies : gisbert fanselow , edith kaan , inge lasser , ming wei lee , michael meng , weijia ni , herbert schriefers and craig thiersch . @ article { bader94 , author = " markus bader " , year = 1994 , title = " syntactic function ambiguities " , journal = " folia linguistica " , volume = " 28 / 1 - - 2 " , pages = " 5 - - 66 " , } @ unpublished { bayer / marslen-wilson 92 , author = " josef bayer and william marslen - wilson " , year = 1992 , title = " configurationality in the light of language comprehension : the order of arguments in { g } erman " , note = " university of leipzig and birbeck university college london " , } @ incollection { crocker94 , author = " matthew w . crocker " , year = 1994 , ti

In [8]:
# b. words corresponding to
vocabulary[40041:40046]

# c. 1 indicates that the corresponding email contains the word

# d. 0 indicates that the corresponding email does not contain the word

['nooteboom', 'nootka', 'nope', 'nor', 'nora']

### 2.2

In [9]:
spam_count = np.count_nonzero(Y)
non_spam_count = np.size(Y) - np.count_nonzero(Y)

# majority classfier accuracy
if spam_count > non_spam_count:
  print(np.mean(Y))
else:
  print(1 - np.mean(Y))

0.8337366055997235


### 2.3

In [10]:
lg_P_S1 = np.log(np.mean(Yt))
lg_P_S0 = np.log(np.mean(~Yt))
lg_P_W1_S1 = np.log(Xt.toarray()[Yt].mean(axis = 0)) 
lg_P_W1_S0 = np.log(Xt.toarray()[~Yt].mean(axis = 0)) 
print('Log Pr(W=1|S=1)', lg_P_W1_S1)
print('Log Pr(W=1|S=0)', lg_P_W1_S0)

<ipython-input-10-bd18f733a0bb>:3: RuntimeWarning: divide by zero encountered in log
  lg_P_W1_S1 = np.log(Xt.toarray()[Yt].mean(axis = 0))


Log Pr(W=1|S=1) [-1.13526561 -1.11939226 -5.96357934 ...        -inf -5.96357934
        -inf]
Log Pr(W=1|S=0) [-1.82288833 -2.89924215 -7.56268125 ... -7.56268125        -inf
        -inf]


<ipython-input-10-bd18f733a0bb>:4: RuntimeWarning: divide by zero encountered in log
  lg_P_W1_S0 = np.log(Xt.toarray()[~Yt].mean(axis = 0))


### 2.4

In [11]:
def compute_prob(content):
    idx = np.where(content == 1)
    tmpt_lg_P_S1 = lg_P_S1 + np.sum(lg_P_W1_S1[np.array(idx)])
    tmpt_lg_P_S0 = lg_P_S0 + np.sum(lg_P_W1_S0[np.array(idx)])
    return [tmpt_lg_P_S0, tmpt_lg_P_S1]
result0 = np.apply_along_axis(compute_prob, 0, Xt.toarray())
result0

array([[       -inf,        -inf,        -inf, ..., -7.74673291,
        -6.64812062, -0.18405166],
       [       -inf,        -inf,        -inf, ...,        -inf,
               -inf, -1.78315356]])

In [12]:
result1 = np.apply_along_axis(compute_prob, 1, Xt.toarray())
result1

array([[ -356.76747718,           -inf],
       [-1430.57828432,           -inf],
       [ -265.00881529,           -inf],
       ...,
       [ -344.03740529,           -inf],
       [ -276.22824202,           -inf],
       [-2478.11318621,           -inf]])

In [13]:
result0.shape

(2, 60925)

In [14]:
result1.shape

(2314, 2)

### 2.5

We have to compute 2 dimension (probabilities) for each email, because we have to compare these probabilities to classify the email as spam or non-spam.

### 2.6

In [15]:
def predict(content):
    idx = np.where(content == 1)
    tmpt_lg_P_S1 = lg_P_S1 + np.sum(lg_P_W1_S1[np.array(idx)])
    tmpt_lg_P_S0 = lg_P_S0 + np.sum(lg_P_W1_S0[np.array(idx)])
    if tmpt_lg_P_S1 > tmpt_lg_P_S0:
        return True
    else:
        return False 

In [16]:
Y_predict = np.apply_along_axis(predict, 1, Xv.toarray())
Y_predict.shape

(579,)

### 2.7

We have need higher the value of the log-likelihood(579), which results in a better model fitting a dataset, and it is useful for comparing two or more models.

### 2.8

In [17]:
def predict(content):
    idx = np.where(content == 1)
    tmpt_lg_P_S1 = lg_P_S1 + np.sum(lg_P_W1_S1[np.array(idx)])
    tmpt_lg_P_S0 = lg_P_S0 + np.sum(lg_P_W1_S0[np.array(idx)])
    return [tmpt_lg_P_S0, tmpt_lg_P_S1]
S0 = np.apply_along_axis(predict, 0, Xt.toarray())
S0

array([[       -inf,        -inf,        -inf, ..., -7.74673291,
        -6.64812062, -0.18405166],
       [       -inf,        -inf,        -inf, ...,        -inf,
               -inf, -1.78315356]])

In [18]:
S1 = np.apply_along_axis(predict, 1, Xt.toarray())
S1

array([[ -356.76747718,           -inf],
       [-1430.57828432,           -inf],
       [ -265.00881529,           -inf],
       ...,
       [ -344.03740529,           -inf],
       [ -276.22824202,           -inf],
       [-2478.11318621,           -inf]])

### 2.9

In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Yv, Y_predict)

array([[487,   0],
       [ 80,  12]])

In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(Yv, Y_predict)

0.8618307426597582

In [21]:
1 - np.mean(Yv)

0.8411053540587219

### 2.10

The cause of such a problem of many infinities was the 0 probability of count of word with regards to few cases of the word, relying on a single rare value for categorization. Thus, we need smoothing methodadding a
small positive number to the counts to caculate probability.

### 3.1 & 3.2

In [29]:
def fitting(Xt, Yt, alpha):
    n_spam = np.sum(Yt) + alpha
    n_non_spam = np.sum(~Yt) + alpha
    n_total = n_spam + n_non_spam
    spam_counts = Xt.toarray()[Yt].sum(0) + alpha
    non_spam_counts = Xt.toarray()[~Yt].sum(0) + alpha
    lg_P_S1 = np.log(n_spam / n_total) 
    lg_P_S0 = np.log(n_non_spam / n_total) 
    lg_P_W1_S1 = np.log(spam_counts / (n_spam + alpha)) 
    lg_P_W1_S0 = np.log(non_spam_counts / (n_non_spam + alpha))

    return lg_P_S1, lg_P_S0, lg_P_W1_S1, lg_P_W1_S0

### 3.3

In [30]:
for alpha in range(5):
    lg_P_S1, lg_P_S0, lg_P_W1_S1, lg_P_W1_S0 = fitting(Xt, Yt, alpha / 10)
    Y_predict = np.apply_along_axis(predict, 1, Xv.toarray())
    print('Alpha:', alpha / 10)
    print('Confusion Matrix:\n', confusion_matrix(Yv, Y_predict))
    print('Accuracy:', accuracy_score(Yv, Y_predict))

<ipython-input-29-794c1d16b53c>:9: RuntimeWarning: divide by zero encountered in log
  lg_P_W1_S1 = np.log(spam_counts / (n_spam + alpha))
<ipython-input-29-794c1d16b53c>:10: RuntimeWarning: divide by zero encountered in log
  lg_P_W1_S0 = np.log(non_spam_counts / (n_non_spam + alpha))


Alpha: 0.0


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 3.4

In [24]:
alphas = np.logspace(-8, 1, num=10)
accuracies = []
for i in alphas:
    lg_P_S1, lg_P_S0, lg_P_W1_S1, lg_P_W1_S0 = fitting(Xt, Yt, i)
    Y_predict = np.apply_along_axis(predict, 1, Xv.toarray())
    accuracies.append(accuracy_score(Yv, Y_predict))
    print('Alpha:', i)
    print('Confusion Matrix:\n', confusion_matrix(Yv, Y_predict))
    print('Accuracy:', accuracy_score(Yv, Y_predict))

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

### 3.5

In [ ]:
import matplotlib.pyplot as plt

_ = plt.plot(np.log10(alphas), accuracies, marker="o")

### 4.1

In [25]:
# probabilities with best accuracy
lg_P_S1, lg_P_S0, lg_P_W1_S1, lg_P_W1_S0 = fitting(Xt, Yt, 0.001)
lg_P_W1 = np.log((np.sum(Xt) + 0.001) / (Xt.shape[0] + 0.002))
lg_P_S1_W1 = lg_P_W1_S1 * lg_P_S1 / lg_P_W1
lg_P_S0_W1 = lg_P_W1_S0 * lg_P_S0 / lg_P_W1
lg_diffs = lg_P_S1_W1 - lg_P_S0_W1
best_idx = np.argsort(-lg_diffs)[:10]
worst_idx = np.argsort(lg_diffs)[:10]
best_vocab = vocabulary[best_idx]
worst_vocab = vocabulary[worst_idx]
(best_vocab, worst_vocab)

TypeError: only integer scalar arrays can be converted to a scalar index

### 4.2

comment:

### 5

In [26]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB

def KFold_CV(X, Y, alpha, k):
    kf = KFold(n_splits=k)
    for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        m = GaussianNB()
        _ = m.fit(X_train, Y_train)
        return m.score(X_train, Y_train)
    
        

In [27]:
alphas = np.logspace(-8, 1, num=10)
accuracies = []
for i in alphas:
  #lg_P_S1, lg_P_S0, lg_P_W1_S1, lg_P_W1_S0 = fitting(Xt, Yt, alpha / 10)
  #Y_predict = np.apply_along_axis(predict, 1, Xv.toarray())
    KFold_accuracy = KFold_CV(X, Y, alpha/10)
    accuracies.append(accuracy_score(Yv, Y_predict))
    print('Accuracy:', KFold_accuracy)

TypeError: KFold_CV() missing 1 required positional argument: 'k'

In [28]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

def cross_val(params):
    model = GaussianNB()
    model.set_params(**params)
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train,
                             cv = 10, #10 folds
                             scoring = "accuracy",
                             verbose = 2
                            )
    #return the mean of the 10 fold cross validation
    return cv_results.mean()

In [ ]:
for X_train, y_train, X_valid, y_valid in k_fold_generator(X, y, k_fold):